# Setup Environment

In [1]:
# Install libraries
!pip install pandas numpy matplotlib seaborn scipy sklearn

In [ ]:
# Importing required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading datasets
# main dataset
df = pd.read_csv('../data/covid_19_data.csv',sep=';')
# region and climate dataset
df_cat = pd.read_csv('../data/covid_19_data_cat.csv')

print("=======================================================")
print(df)
print("=======================================================")
print(df.info())

# Dataset Information

In [ ]:
# No. of columns and rows
print("Number of Columns:", len(df.columns))
print("Number of Rows:", len(df))

# No. of countries
print("Number of Countries:", len(df.country.unique()))

# Convert data type
df['date'] = df['date'].astype('datetime64[ns]')
df['date'] = pd.to_datetime(df['date'], format='%y%m%d').dt.date

# No. of days
print("Number of Days:", len(df.date.unique()))
print("Since:", min(df.date))
print("Until:", max(df.date))

#### Replace some values in the 'country' column so that there are no different names for the same country:
'original data','replace with this data':
<br>
"('St. Martin',)",'St. Martin'
<br>
' Azerbaijan','Azerbaijan'
<br>
'Cabo Verde','Cape Verde'
<br>
'Congo (Brazzaville)','Congo'
<br>
'Congo (Kinshasa)','Congo'
<br>
'North Ireland','Ireland'
<br>
'North Macedonia','Macedonia'
<br>
'occupied Palestinian territory','Palestine'
<br>
'Holy See','Vatican'
<br>
'Republic of Ireland','Ireland'
<br>
'The Bahamas','Bahamas'
<br>
'The Gambia','Gambia'
<br>
'Bahamas, The','Bahamas'
<br>
'Gambia, The','Gambia'
<br>
'Vatican City','Vatican'
<br>
'East Timor','Timor-Leste'
<br>
'West Bank and Gaza','Palestine'
<br>
'MS Zaandam','Others'
<br>
'Diamond Princess','Others'

In [ ]:
# replace with this data
replace_country = {"('St. Martin',)":'St. Martin',
                   ' Azerbaijan':'Azerbaijan',
                   'Cabo Verde':'Cape Verde',
                   'Congo (Brazzaville)':'Congo',
                   'Congo (Kinshasa)':'Congo',
                   'North Ireland':'Ireland',
                   'North Macedonia':'Macedonia',
                   'occupied Palestinian territory':'Palestine',
                   'Holy See':'Vatican',
                   'Republic of Ireland':'Ireland',
                   'The Bahamas':'Bahamas',
                   'The Gambia':'Gambia',
                   'Bahamas, The':'Bahamas',
                   'Gambia, The':'Gambia',
                   'Vatican City':'Vatican',
                   'East Timor':'Timor-Leste',
                   'West Bank and Gaza':'Palestine',
                   'MS Zaandam':'Others',
                   'Diamond Princess':'Others'
                  }

df = df.replace({"country": replace_country})

In [ ]:
# New no. of countries
print("Number of Countries:", len(df.country.unique()))

####  Several 'countries' on the 'df' dataframe have daily data divided into several 'provinces'. Accumulate the 'confirmed', 'deaths' and 'recovered' data for these provinces so that the daily data for each country is only represented by one row by creating a new dataframe 'df_new'.

In [ ]:
print("Daily Data for Each Country")
df_new = df.groupby(by=['country', 'date']).sum().reset_index()
print(df_new)

#### Drop all rows in 'df_new' which data 'confirmed' is below 100

In [ ]:
print("Daily Data for Each Country Which Confimed >= 100")
df_new = df_new[df_new['confirmed'] >= 100]
print(df_new)

#### Add 'region' and 'climate' columns to 'df_new' and fill in the region and climate for each country by referring to 'df_cat' 

In [ ]:
print("Add 'region' and 'climate' columns")
df_new = df_new.merge(df_cat, how='left', on="country")
print(df_new)
df_new.to_csv('covid-aug-20.csv')

#### Create a line plot based on dataframe 'df_new' with data 'date' as x and data 'confirmed' as y, where each line represents the accumulative data of each region 

In [ ]:
# get data for each region
df_asipac = df_new[df_new['region'] == 'Asia & Pacific'].groupby(by=['date']).sum().reset_index()
df_europe = df_new[df_new['region'] == 'Europe'].groupby(by=['date']).sum().reset_index()
df_arab = df_new[df_new['region'] == 'Arab States'].groupby(by=['date']).sum().reset_index()
df_africa = df_new[df_new['region'] == 'Africa'].groupby(by=['date']).sum().reset_index()
df_latin = df_new[df_new['region'] == 'South/Latin America'].groupby(by=['date']).sum().reset_index()
df_northam = df_new[df_new['region'] == 'North America'].groupby(by=['date']).sum().reset_index()
df_mideast = df_new[df_new['region'] == 'Middle east'].groupby(by=['date']).sum().reset_index()

# plot based on date
plt.plot(df_asipac["date"], df_asipac["confirmed"], label="Asia & Pacific")
plt.plot(df_europe["date"], df_europe["confirmed"], label="Europe")
plt.plot(df_arab["date"], df_arab["confirmed"], label="Arab States")
plt.plot(df_africa["date"], df_africa["confirmed"], label="Africa")
plt.plot(df_latin["date"], df_latin["confirmed"], label="South/Latin America")
plt.plot(df_northam["date"], df_northam["confirmed"], label="North America")
plt.plot(df_mideast["date"], df_mideast["confirmed"], label="Middle East")
plt.legend(loc='upper left')
plt.title("Data Covid-19 Based on Region from Jan to Aug")
plt.show()

#### Create a line plot based on dataframe 'df_new' with data 'date' as x and data 'confirmed' as y, where each line represents the accumulative data of each climate 

In [ ]:
# get data for each climate
df_nontropic = df_new[df_new['climate'] == 'nontropic'].groupby(by=['date']).sum().reset_index()
df_tropic = df_new[df_new['climate'] == 'tropic'].groupby(by=['date']).sum().reset_index()

# plot based on date
plt.plot(df_nontropic["date"], df_nontropic["confirmed"], label="nontropic")
plt.plot(df_tropic["date"], df_tropic["confirmed"], label="tropic")
plt.legend(loc='upper left')
plt.title("Data Covid-19 Based on Climate from Jan to Aug")
plt.show()

#### Create a dataframe 'df_last' that only contains data from the last date of 'df_new', where each row shows data for 'confirmed', 'deaths', 'recovered', 'region', and 'climate' for each country. 

In [ ]:
# get last updated data for each country
print("Last Updated Data for Each Country")
df_last = df_new.groupby(by=["country"]).max().reset_index()
print(df_last)
df_last.to_csv('last_update_covid.csv')

####  Based on the df last, identify the top 10 countries with the highest data for 'deaths' and then create the barplot.

In [ ]:
# identify the top 10 countries with the highest number of deaths
df_10 = df_last.sort_values(by=['deaths'], ascending=False).head(10)
print(df_10)

# creating the bar plot 
plt.bar(df_10["country"], df_10["deaths"], color ='maroon', width = 0.4) 
  
plt.xlabel("Countries") 
plt.ylabel("No. of deaths") 
plt.title("Top 10 Negara dengan Death Tertinggi") 
plt.show()

#### Perform EDA on the 'df_last' dataframe for the 'confirmed', 'deaths' and 'recovered' columns using a scatter matrix (distinguish the scatter plot colors by region) 

In [ ]:
# import library
import seaborn as sns
sns.set_theme(style="ticks")

# create a scatter matrix for region
ax = sns.pairplot(df_last, hue="region")
ax.fig.suptitle("Scatter Matrix By Region", y=1.08)

#### Perform EDA on the 'df_last' dataframe for the 'confirmed', 'deaths' and 'recovered' columns using a scatter matrix (distinguish the scatter plot colors by climate) 

In [ ]:
sns.set_theme(style="ticks")

ax= sns.pairplot(df_last, hue="climate")
ax.fig.suptitle("Scatter Matrix Berdasarkan Climate", y=1.08)

####  Based on the results in 4.c. and d., what data do visually appear to have a normal distribution?

None

#### Confirm the answer to question 5.a. by using the normality test from the scipy module. 

In [ ]:
# import library
from scipy.stats import normaltest

# alpha value
alpha = 0.05

# 'confirmed' normality test
print("====================confirmed==========================")
stat, p = normaltest(df_last["confirmed"])
print('Statistics=%.3f, p=%.3f' % (stat, p))
if p > alpha:
    print('Normal Distribution (fail to reject H0)')
else:
    print('Not Normal Distribution (reject H0)')
    
# 'deaths' normality test
print("======================deaths===========================")
stat, p = normaltest(df_last["deaths"])
print('Statistics=%.3f, p=%.3f' % (stat, p))
if p > alpha:
    print('Normal Distribution (fail to reject H0)')
else:
    print('Not Normal Distribution (reject H0)')
    
# 'recovered' normality test
print("=====================recovered=========================")
stat, p = normaltest(df_last["recovered"])
print('Statistics=%.3f, p=%.3f' % (stat, p))
if p > alpha:
    print('Normal Distribution (fail to reject H0)')
else:
    print('Not Normal Distribution (reject H0)')

#### Perform an independent t-test to test whether or not a correlation between 'climate' and 'confirmed' in 'df_last'.

In [ ]:
# import library
from scipy import stats

# convert data type 'climate'
df_last["climate"] = df_last["climate"].astype('category')
df_last["climate_cat_codes"] = df_last["climate"].cat.codes

# perform independent t-test
stat, p = stats.ttest_ind(df_last['confirmed'], df_last['climate_cat_codes'])

# hypothesis
if p > alpha:
    print('fail to reject H0')
else:
    print('reject H0')

#### If alpha value = 0.05, can the H0 that 'there is no correlation between the climate group and the' Confirmed 'data be rejected? What is the reason?

There is a correlation between the climate groups and the data confirmed because the p-value was smaller than the alpha value

#### Build a classifier model to predict region of 'X_new' based on the 'confirmed', 'deaths' and 'recovered' data contained in 'df_last'. Show the accuracy of the model using a train-test split.

In [ ]:
# import library
import os
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz

# path graphviz
os.environ["PATH"] += os.pathsep + 'C:\\Users\\kevin\\anaconda3\\Library\\bin\\graphviz'

# drop na if exists
df_last = df_last.dropna()

# train-test split
X = df_last.iloc[:,2:5]
y = df_last["region"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# modeling
clf = RandomForestClassifier(n_estimators=100, max_features="auto", max_depth=None, 
                             min_samples_split=2, min_samples_leaf=1, bootstrap=True)
clf = clf.fit(X_train, y_train)

# predict test data
y_pred = clf.predict(X_test)

# scoring
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
# # import library
# from sklearn.model_selection import GridSearchCV

# # parameter grid
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]

# parameters = {"n_estimators": n_estimators,
#               "max_features": max_features,
#               "min_samples_split": max_depth,
#               "min_samples_leaf": min_samples_split,
#               "bootstrap": bootstrap
#              }

# # parameter optimization
# gs = GridSearchCV(clf, param_grid=parameters, cv=2, verbose=1, n_jobs=-1)
# gs.fit(X, y)

# # best parameter
# print("Best Param:", gs.best_params_)
# print("Best Estimator:", gs.best_estimator_)
# print("Best Score:", gs.best_score_)

In [ ]:
# extract single tree
estimator = clf.estimators_[0]
print(estimator)

# visualizing
fn = ["confirmed", "deaths", "recovered"]
cn = ['AP', 'EU', 'Arab', 'Afr', 'Latin','NA', 'ME']

dot_data = export_graphviz(estimator, out_file=None,
                           feature_names=fn,
                           class_names=cn,
                           filled=True, rounded=True,  
                           special_characters=True)  
graph = graphviz.Source(dot_data)  
graph
# graph.render('dtree_region',view=True)

In [ ]:
# predict new data
X_new=np.array([[1000,30,200],
                [2000,40,400],
                [50,1,2]])

y_pred = clf.predict(X_new)
print(y_pred)

In [ ]:
# get importance
importance = clf.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

#### Build a classifier model to predict climate of 'X_new' based on the 'confirmed', 'deaths' and 'recovered' data contained in 'df_last'. Show the accuracy of the model using a train-test split.

In [ ]:
# train-test split
X = df_last.iloc[:,2:5]
y = df_last["climate"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# modeling
clf = RandomForestClassifier(n_estimators=100, max_features="auto", max_depth=None, 
                             min_samples_split=2, min_samples_leaf=1, bootstrap=True)
clf = clf.fit(X_train, y_train)

# predict test data
y_pred = clf.predict(X_test)

# scoring
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
# # import library
# from sklearn.model_selection import GridSearchCV

# # parameter grid
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]

# parameters = {"n_estimators": n_estimators,
#               "max_features": max_features,
#               "min_samples_split": max_depth,
#               "min_samples_leaf": min_samples_split,
#               "bootstrap": bootstrap
#              }

# # parameter optimization
# gs = GridSearchCV(clf, param_grid=parameters, cv=5, verbose=1, n_jobs=-1)
# gs.fit(X, y)

# # best parameter
# print("Best Param:", gs.best_params_)
# print("Best Estimator:", gs.best_estimator_)
# print("Best Score:", gs.best_score_)

In [ ]:
# extract single tree
estimator = clf.estimators_[0]
print(estimator)

# visualizing
fn = ["confirmed", "deaths", "recovered"]
cn = ['nontropic', 'tropic']

dot_data = export_graphviz(estimator, out_file=None,
                           feature_names=fn,
                           class_names=cn,
                           filled=True, rounded=True,  
                           special_characters=True)  
graph = graphviz.Source(dot_data)  
graph
# graph.render('dtree_climate',view=True)

In [ ]:
# predict new data
X_new=np.array([[1000,30,200],
                [2000,40,400],
                [50,1,2]])

y_pred = clf.predict(X_new)
print(y_pred)

In [ ]:
# get importance
importance = clf.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

#### Build a linear regression model for the distribution of the number of deaths in the US from 20 March 2020 - 10 August 2020. Plot this regression model. Calculate the R^2 and RMSE values of the model using a train-test split.

In [ ]:
# import library
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# select date ranges
df_lr = df.set_index('date')
startdate = pd.to_datetime("2020-03-20").date()
enddate = pd.to_datetime("2020-08-10").date()
df_lr = df_lr.loc[startdate : enddate].reset_index()

# train-test split
X = df_lr[['confirmed']]
y = df_lr["deaths"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# modeling
regressor = LinearRegression()
regressor.fit(X_train, y_train)
print("Intercept:", regressor.intercept_)
print("Coefficient:", regressor.coef_)

# predict test data
y_pred = regressor.predict(X_test)
df_pred = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# scoring
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R^2 Score:', r2_score(y_test, y_pred))

# visualizing
plt.scatter(X_test, y_test)
plt.plot(X_test, y_pred, color='red')
plt.show()

#### Build a clustering model with 5 clusters (cluster 0-4) for the 'Z' array. Predict the cluster numbers based on 'confirmed', 'deaths' and 'recovered' data for 'df_last' for the following countries:
<br>
a. Indonesia 4
<br>
b. Singapore 4
<br>
c. US 1
<br>
d. Italy 4
<br>
e. Iran 4

In [ ]:
# import library
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# data
Z=df_last.loc[:,['confirmed','recovered','deaths']].values
Z[:5]

# normalization
scaler = StandardScaler()
scaled_features = scaler.fit_transform(Z)

# modeling
kmeans = KMeans(init="random", n_clusters=5, n_init=10, max_iter=300, random_state=42)
kmeans.fit(scaled_features)
print(kmeans.labels_[:5])

# scoring
kmeans_silhouette = silhouette_score(scaled_features, kmeans.labels_).round(2)
print("Silhouette Score:", kmeans_silhouette)

# predict new data
df_indonesia = df_last.loc[df_last['country'] == "Indonesia"].iloc[:,2:5]
print(df_indonesia)

df_singapore = df_last.loc[df_last['country'] == "Singapore"].iloc[:,2:5]
print(df_singapore)

df_us = df_last.loc[df_last['country'] == "US"].iloc[:,2:5]
print(df_us)

df_italy = df_last.loc[df_last['country'] == "Italy"].iloc[:,2:5]
print(df_italy)

df_iran = df_last.loc[df_last['country'] == "Iran"].iloc[:,2:5]
print(df_iran)

df_new_pred_cluster = pd.concat([df_indonesia, df_singapore, df_us, df_italy, df_iran])

X = StandardScaler().fit_transform(df_new_pred_cluster)
y_pred = kmeans.predict(X)
print(y_pred)